food recommendation

github: https://github.com/chf-NewStart/someUsefulStuff/

## API Book Keeping

In [ ]:
import json
import requests

# Replace 'your_new_api_key' with your actual API key
api_key_data = {
    "yelp_api_key" : "your_yelp_api_key",
    "gmap_api_key" : "your_gmap_api_key",
    "openai_api_key" : "your_openai_api_key"

}

# Save the API key in a JSON file
with open("api_key.json", "w") as json_file:
    json.dump(api_key_data, json_file)

print("API key saved to api_key.json")


# Load the API key from the JSON file
with open("api_key.json", "r") as json_file:
    key_data = json.load(json_file)

# Get the API key from the dictionary
yelp_api_key = key_data['yelp_api_key']
gmap_api_key = key_data['gmap_api_key']
openai_api_key = key_data['openai_api_key']

API key saved to api_key.json


In [ ]:
!pip install openai


## Use LLM to translate desired flavor into types of restaurants

In [ ]:
import openai

# Set up OpenAI API key
openai.api_key = openai_api_key

def refine_search_query(user_input):
    # Use the LLM to interpret the user’s flavor/taste preferences
    response = openai.chat.completions.create(
        model="gpt-4",
         messages=[
            {"role": "system", "content": "You are an assistant who helps refine food-related search queries based on user preferences."},
            {"role": "user", "content": f"Given the user preference '{user_input}', suggest two specific type of food or cuisine that can be used in Yelp or Google Maps searches."}
        ],
        max_tokens=50,
    )
    # Extract the suggested query from the LLM response
    print(response)
    refined_query = response.choices[0].message.content.strip()
    return refined_query


In [ ]:
# Accept user input for flavor or taste preference
user_input = input("What kind of flavor or taste are you craving? ")

# Process the input with LLM to refine the search term
refined_query = refine_search_query(user_input)
print(f"Refined Query from LLM: {refined_query}")


What kind of flavor or taste are you craving? spicy and sour
ChatCompletion(id='chatcmpl-APcnXURLLN2ijLjOWGaHLF8iSm0kW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. "Szechuan cuisine"\n2. "Thai Tom Yum soup"', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1730670703, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=56, total_tokens=73, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))
Refined Query from LLM: 1. "Szechuan cuisine"
2. "Thai Tom Yum soup"


## Use yelp and google maps and generate recommendations

In [ ]:

import requests

# Yelp search function
def search_yelp(query, location="Toronto"):
    yelp_url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization": f"Bearer {yelp_api_key}"}
    params = {"term": query, "location": location, "limit": 5}
    response = requests.get(yelp_url, headers=headers, params=params)
    return response.json()

# Google Maps search function
def search_google_maps(query, location="Toronto"):
    gmap_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {"query": f"{query} restaurants in {location}", "key": gmap_api_key}
    response = requests.get(gmap_url, params=params)
    return response.json()


def format_gmap_results(gmap_data):
    recommendations = []
    if gmap_data and 'results' in gmap_data:
        for place in gmap_data['results']:
            recommendations.append({
                'Name': place.get('name'),
                'Rating': place.get('rating'),
                'Reviews': place.get('user_ratings_total', 'N/A'),
                'Address': place.get('formatted_address'),
                'Google Maps URL': f"https://www.google.com/maps/place/?q=place_id:{place['place_id']}"
            })
    return recommendations

# Perform searches with the refined query


# Format Yelp results with error handling
def format_yelp_results(yelp_data):
    recommendations = []
    if yelp_data and 'businesses' in yelp_data:  # Ensure data is valid
        for business in yelp_data['businesses']:
            recommendations.append({
                'Name': business.get('name'),
                'Rating': business.get('rating'),
                'Reviews': business.get('review_count'),
                'Categories': ', '.join([cat['title'] for cat in business.get('categories', [])]),
                'Address': ', '.join(business['location']['display_address']),
                'Phone': business.get('phone', 'N/A'),
                'URL': business.get('url')
            })
    else:
        print("No valid Yelp data to format.")
    return recommendations

# Sample usage with error handling
yelp_results = search_yelp(refined_query)
yelp_recommendations = format_yelp_results(yelp_results)

gmap_results = search_google_maps(refined_query)
gmap_recommendations = format_gmap_results(gmap_results)


## Combine and phrase the recommendations

In [ ]:
# Assuming yelp_recommendations and gmap_recommendations are already defined

# Combine the results from Yelp and Google Maps
combined_recommendations = yelp_recommendations + gmap_recommendations

# Now, display the recommendations
for i, recommendation in enumerate(combined_recommendations, start=1):
    print(f"Recommendation #{i}:")
    print(f"Name: {recommendation['Name']}")
    print(f"Rating: {recommendation['Rating']} ({recommendation['Reviews']} reviews)")
    print(f"Categories/Type: {recommendation.get('Categories', 'N/A')}")
    print(f"Address: {recommendation['Address']}")
    print(f"Phone: {recommendation.get('Phone', 'N/A')}")
    print(f"URL: {recommendation.get('URL', recommendation.get('Google Maps URL', 'N/A'))}")
    print("=" * 50)


Recommendation #1:
Name: A Szechuan Restaurant
Rating: 4.4 (452 reviews)
Categories/Type: N/A
Address: 440 Spadina Ave., Toronto, ON M5T 2G8, Canada
Phone: N/A
URL: https://www.google.com/maps/place/?q=place_id:ChIJg5xXzUQ1K4gR9NHKVt3hTRc
Recommendation #2:
Name: Si Lom Thai Bistro (ร้านสีลม โตรอนโต้)
Rating: 4.4 (1447 reviews)
Categories/Type: N/A
Address: 534 Church St, Toronto, ON M4Y 2E1, Canada
Phone: N/A
URL: https://www.google.com/maps/place/?q=place_id:ChIJv7nEMbM0K4gRo7GUIuPuBI8
Recommendation #3:
Name: Sichuan Garden Restaurant
Rating: 3.8 (257 reviews)
Categories/Type: N/A
Address: 359 Spadina Ave., Toronto, ON M5T 2G3, Canada
Phone: N/A
URL: https://www.google.com/maps/place/?q=place_id:ChIJ3RWlYcE0K4gRSyRlCGVEtgE
Recommendation #4:
Name: Szechuan Noodle Bowl
Rating: 4.7 (230 reviews)
Categories/Type: N/A
Address: 526 Bloor St W, Toronto, ON M5S 1Y3, Canada
Phone: N/A
URL: https://www.google.com/maps/place/?q=place_id:ChIJ98vtos81K4gRrVFihB1kCd4
Recommendation #5:
Name: Yue

In [ ]:
import pandas as pd

# Convert combined recommendations to a DataFrame
df = pd.DataFrame(combined_recommendations)

# Save as CSV file
df.to_csv("combined_recommendations.csv", index=False)

print("Recommendations saved to combined_recommendations.csv")


Recommendations saved to combined_recommendations.csv
